# 02 - Címke Elemzés (Label Analysis)
A Label Studio-ból exportált `labels.json` feldolgozása. Megvizsgáljuk, hány annotáció van, milyen az eloszlásuk típusok szerint (Normal, Pennant, Wedge), és milyen hosszúak ezek az alakzatok.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import numpy as np

# Útvonalak
DATA_DIR = "../data" if os.path.exists("../data") else "/app/data"

# Minden labels.json megkeresése
label_files = glob.glob(os.path.join(DATA_DIR, "**", "labels.json"), recursive=True)
print(f"Talált címke fájlok: {len(label_files)}")

all_data = []
for lf in label_files:
    try:
        with open(lf, 'r') as f:
            tasks = json.load(f)
            print(f" - {os.path.relpath(lf, DATA_DIR)}: {len(tasks)} task")
            all_data.extend(tasks)
    except Exception as e:
        print(f"Hiba {lf}: {e}")

print(f"Összesen {len(all_data)} annotált task.")

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import numpy as np

# Útvonalak
DATA_DIR = "../data" if os.path.exists("../data") else "/app/data"

# Minden labels.json megkeresése
label_files = glob.glob(os.path.join(DATA_DIR, "**", "labels.json"), recursive=True)
print(f"Talált címke fájlok: {len(label_files)}")

all_data = []
for lf in label_files:
    try:
        with open(lf, 'r') as f:
            tasks = json.load(f)
            print(f" - {os.path.relpath(lf, DATA_DIR)}: {len(tasks)} task")
            all_data.extend(tasks)
    except Exception as e:
        print(f"Hiba {lf}: {e}")

print(f"Összesen {len(all_data)} annotált task.")

In [ ]:
stats = []

for task in all_data:
    filename = task.get('file_upload', 'Unknown')
    
    for ann in task.get('annotations', []):
        for res in ann.get('result', []):
            val = res.get('value', {})
            labels = val.get('timeserieslabels', [])
            
            if labels:
                label_name = labels[0]
                start = pd.to_datetime(val['start'])
                end = pd.to_datetime(val['end'])
                
                # Hossz számítása
                duration_mins = (end - start).total_seconds() / 60
                
                # Feltételezzük, hogy az adatok 5 percesek (a fájlnevek alapján általában)
                # Ha kevert (1h, 15m), akkor ez a becslés pontatlan lehet, 
                # de a feladat szerint a "darabszám" a cél.
                candle_count = int(duration_mins / 5)
                if candle_count < 1: candle_count = 1
                
                stats.append({
                    'File': filename,
                    'Label': label_name,
                    'Candle_Count': candle_count,
                    'Duration_Min': duration_mins
                })

df_stats = pd.DataFrame(stats)
print(f"Összes annotáció (Label): {len(df_stats)}")
df_stats.head()

In [ ]:
plt.figure(figsize=(10, 6))
ax = df_stats['Label'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')

plt.title("Annotációk eloszlása típusonként (Összesített)")
plt.xlabel("Alakzat típusa")
plt.ylabel("Darabszám")
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 5), textcoords='offset points')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df_stats['Candle_Count'], bins=20, color='orange', edgecolor='black')

plt.title("Alakzatok hossza (Gyertyákban, 5p alap)")
plt.xlabel("Hossz (db gyertya)")
plt.ylabel("Gyakoriság")
plt.grid(axis='y', alpha=0.3)
plt.show()

print("Átlagos hossz:", df_stats['Candle_Count'].mean())
print("Min:", df_stats['Candle_Count'].min())
print("Max:", df_stats['Candle_Count'].max())